In [8]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [9]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [10]:
df_cq_tr = pd.read_csv(generaPath('O', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('O', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('O', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('O', 'df_cc_re.csv'))

df_ct_tr = pd.read_csv(generaPath('O', 'df_ct_tr.csv'))
df_ct_re = pd.read_csv(generaPath('O', 'df_ct_re.csv'))

df_cd_tr = pd.read_csv(generaPath('O', 'df_cd_tr.csv'))
df_cd_re = pd.read_csv(generaPath('O', 'df_cd_re.csv'))

df_cs_tr = pd.read_csv(generaPath('O', 'df_cs_tr.csv'))
df_cs_re = pd.read_csv(generaPath('O', 'df_cs_re.csv'))

df_cs_tr_sb = pd.read_csv(generaPath('O', 'df_cs_tr_sb.csv'))
df_cs_re_sb = pd.read_csv(generaPath('O', 'df_cs_re_sb.csv'))


df_sel_sb_tr = pd.read_csv(generaPath('O','df_sel_sb_tr.csv'))
df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_sb_re.csv'))

df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_si_tr.csv'))
df_sel_si_re = pd.read_csv(generaPath('O','df_sel_si_re.csv') )

sbs = pd.read_csv(generaPath('O','sbs.csv'))
singles = pd.read_csv(generaPath('O','singles.csv'))



In [11]:
def unsplited_tupla(secuence, colon):
    partido = secuence.split(sep=colon) if (secuence.find(colon) >= 1) else secuence
    return partido

In [12]:
def crea_sorteos(single,sb, secuence):
    sorteo = []
    enteros = []
    colon = ','
    #splitedtupla = unsplited_tupla(secuence, colon)
    
    #print(splitedtupla)
    sorteo = singles.iloc[secuence, 1]
    print(type(sorteo))
    #sorteo.append(str(sb))
    return sorteo

In [13]:
def cayo_antestetos(sbs, singles):
    sorteos = []
    modaa = []
    lasts = []
    delta = []  
    resta = []
    secuencia = [1,2,3,4,5]
    char = ','
    for index, row in sbs.iterrows():
        sb = str(row[0])
        #print(sb)
        sorteos.append(crea_sorteos(singles,sb, secuencia))

        #
        #print(dftx)
        #df_filt = dftx.iloc[0::1,[0]]
        #df_tupla = calcular_deltas_verticales(df_filt)
        #lasts.append(df_tupla[0])
        #media.append(calc_media(df_tupla))
        #resta.append(abs(int(df_tupla[0])- int(calc_media(df_tupla))))
        #df_moda = calc_moda(df_tupla)
        #pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        #modaa.append(pal)
        #palabra = '|'.join([str(e) for e in df_tupla]) 
        #delta.append(palabra)
    dft = sorteos
    #proba, proba1,proba2,proba3 = how_hot_is(dft, df_tr_fu_filt_delta)
    #dft['Pr']= proba
    #dft['PrX'] = proba1
    #dft['PrL'] = proba2
    #dft['PrM'] = proba3
    return dft

In [14]:
cayo_antestetos(sbs, singles)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


[1    1
 2    1
 3    1
 4    1
 5    1
 Name: R, dtype: int64, 1    1
 2    1
 3    1
 4    1
 5    1
 Name: R, dtype: int64, 1    1
 2    1
 3    1
 4    1
 5    1
 Name: R, dtype: int64]